<a href="https://colab.research.google.com/github/linbinbin/Langchain_practice/blob/main/colab/Langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install openai
!pip install tiktoken
!pip install langchain-community
!pip install langgraph
!pip install langchain-anthropic
!pip install tavily-python

In [ ]:
import getpass
import os
from google.colab import userdata
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

#os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["ANTHROPIC_API_KEY"] = userdata.get('CLAUDE_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = "claude_01"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"


model = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=4096,
    timeout=None,
    max_retries=3,
    # other params...
)

#model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "agent_thread_id_1"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': 'Okay, nice to meet you Bob! Since you mentioned that you live in SF (San Francisco), I can try to provide some helpful information about the city:', 'type': 'text'}, {'id': 'toolu_01Bg6bdxQWssYGcrMzQwzHJm', 'input': {'query': 'san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01FsAbND2KYok4RzdMGNnN1o', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 376, 'output_tokens': 92}}, id='run-91aaa43d-aa2a-4906-8bac-342e6baf0e9c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'san francisco'}, 'id': 'toolu_01Bg6bdxQWssYGcrMzQwzHJm'}], usage_metadata={'input_tokens': 376, 'output_tokens': 92, 'total_tokens': 468})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.sftravel.com/", "content": "Richmond / Presidio\\nThe southern anchor of the Golden Gate Bridge, the verdant, sce

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Jim")], "language": "Japanese"},
    config=config,
)

response.content

'こんにちは、Jimさん。私はあなたの質問に日本語で答えさせていただきます。どのようなことでも質問してください。'

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

response.content

'申し訳ありませんが、私はあなたの名前がJimさんだと理解しています。私の記憶では、最初に自己紹介の際にJimさんと仰っていただきました。私は間違っていますでしょうか?'

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm job")], "language": "Chinese"}
)

response.content

'你好,很高兴认识你!我是一个人工智能助手,我会尽我所能回答你的问题。请问你有什么需要帮助的吗?'